In [1]:
from poitousprint import Portic, Toflit
import json
import networkx as nx
from ipysigma import Sigma

portic_client = Portic()
toflit_client = Toflit()

# this function allows to map a value from a domain of min-max to another
def map_value(value, domain_min, domain_max, range_min, range_max):
    left_span = domain_max - domain_min
    right_span = range_max - range_min

    # Convert the left range into a 0-1 range (float)
    scaled = float(value - domain_min) / float(left_span)

    # Convert the 0-1 range into a value in the right range.
    return range_min + (scaled * right_span)

#### Fonction générique qui permet de fabriquer un réseau à partir :

1. d'une liste de dicts (ex. flux toflit18)
2. d'une liste de deux propriétés à comparer
3. d'un param filter_import_export que l'on peut mettre sur import ou exports (par défaut on a à la fois les imports et les exports

In [20]:
def render_coocurrences_graph(data, key_1, key_2, filter_export_import=None, params=None):
    if filter_export_import not in [None, 'Exports', 'Imports']:
        print("filter_export_import must be 'Exports', 'Imports' or None")
        raise
    
    # créer un graphe
    Graph = nx.Graph()

    # créer des dict pour les deux types de noeuds et les liens
    key1_uniq = {}
    key2_uniq = {}
    edges_uniq = {}
    default_params = {
        "color_1": "rgb(0, 255, 0)",
        "color_2": "rgb(255, 0, 0)",
        "node_min_size": 1,
        "node_max_size": 10
    }
    final_params = default_params
    if params is not None :
        final_params = {
            *default_params,
            *params
        }
    
    # remplir les dicts
    for datum in data:
        if (filter_export_import is not None and datum['export_import'] != filter_export_import):
            continue
            
        if key_1 in datum and key_2 in datum:
            value_1 = datum[key_1] if datum[key_1] is not None else "undefined"
            value_2 = datum[key_2] if datum[key_2] is not None else "undefined"
            value_1_id = key_1 + "_" + value_1
            value_2_id = key_2 + "_" + value_2
            
            if value_1_id in key1_uniq:
                key1_uniq[value_1_id] = {**key1_uniq[value_1_id], "size": key1_uniq[value_1_id]["size"] + 1}
            else:
               key1_uniq[value_1_id] = {
                   "type": key_1, 
                   "name": value_1, 
                   "color": final_params["color_1"],
                   "size": 1
               }
            
            if value_2_id in key2_uniq:
                key2_uniq[value_2_id] = {**key2_uniq[value_2_id], "size": key2_uniq[value_2_id]["size"] + 1}
            else:
               key2_uniq[value_2_id] = {
                   "type": key_2, 
                   "name": value_2, 
                   "color": final_params["color_2"],
                   "size": 1
               }
            
            edge_footprint = value_1_id + "-" + value_2_id
            if edge_footprint in edges_uniq:
                edges_uniq[edge_footprint]["weight"] += 1
            else:
                edges_uniq[edge_footprint] = {
                    "source": value_1_id,
                    "target": value_2_id,
                    "weight": 1
                }
                
    # concaténer les deux dicts de noeuds en un seul
    all_nodes = key1_uniq
    all_nodes.update(key2_uniq)
    # applatir et formatter les noeuds
    nodes = []
    for key, node in all_nodes.items():
        nodes.append((key, node))
    edges = []

    for key, edge in edges_uniq.items():
        edges.append((edge["source"], edge["target"], {"weight": edge["weight"]}))
        
    # ajuster la taille des noeuds en fonction d'un min et d'un max donnés
    domain_min_nodes_size = min([node[1]['size'] for node in nodes])
    domain_max_nodes_size = max([node[1]['size'] for node in nodes])
    range_in_nodes_size = [final_params["node_min_size"], final_params["node_max_size"]]
    nodes_size_mapping_params = [domain_min_nodes_size, domain_max_nodes_size, *range_in_nodes_size]

    for node in nodes:
        node[1]["size"] = map_value(node[1]["size"], *nodes_size_mapping_params)
        node[1]["label"] = node[1]["name"]


    Graph.add_nodes_from(nodes)
    Graph.add_edges_from(edges)

    return Sigma(Graph, start_layout=True)

# 0. Je vais chercher les flux du sprint (côté Toflit) pour nourrir mes réseaux

nb : pour les flux concernant le sprint on n'a qu'une seule direction des Fermes : La Rochelle

In [23]:
# je vais chercher les flux qui concernent le sprint côté Toflit

flows = toflit_client.get_flows(
    year=1789,
    customs_region='La Rochelle', 
    params=[
      "product_revolutionempire",
      "partner",
      "export_import",
      "value",
      "line",
      "partner_simplification",
      "customs_office",
      "customs_region"
	]
)
flows[0]    

{'customs_region': 'La Rochelle',
 'partner': 'Petites iles',
 'export_import': 'Exports',
 'value': '15120',
 'customs_office': "Les Sables d'Olonne",
 'partner_simplification': 'Petites Îles',
 'product_revolutionempire': 'Animaux'}

In [24]:
len(flows)

1062

# 1. Réseau bipartite entre les directions des Fermes et les partenaires commerciaux

In [21]:
render_coocurrences_graph(flows, "customs_region", "partner_simplification")

Sigma(data={'nodes': [('customs_region_La Rochelle', {'type': 'customs_region', 'name': 'La Rochelle', 'color'…

# 2. Réseau bipartite entre les bureaux des Fermes et les partenaires commerciaux 

## A. avec filtre sur les imports

In [25]:
render_coocurrences_graph(flows, "customs_office", "partner_simplification", filter_export_import = 'Imports')

Sigma(data={'nodes': [("customs_office_Les Sables d'Olonne", {'type': 'customs_office', 'name': "Les Sables d'…

## B. avec filtre sur les exports

In [26]:
render_coocurrences_graph(flows, "customs_office", "partner_simplification", filter_export_import = 'Exports')

Sigma(data={'nodes': [("customs_office_Les Sables d'Olonne", {'type': 'customs_office', 'name': "Les Sables d'…

# 3. Réseau tripartite entre les bureaux des Fermes, les produits et les partenaires commerciaux

In [30]:
# je customise la fonction de création de Graphe

def render_tripartite_graph(data, key_1, key_2, key_3, filter_export_import=None, params=None):
    if filter_export_import not in [None, 'Exports', 'Imports']:
        print("filter_export_import must be 'Exports', 'Imports' or None")
        raise
    
    # créer un graphe
    Graph = nx.Graph()

    # créer des dict pour les deux types de noeuds et les liens
    key1_uniq = {} # -> products
    key2_uniq = {} # -> partners
    key3_uniq = {} # customs->office
    edges_uniq = {}
    
    default_params = {
        "color_1": "rgb(0, 255, 0)", # noeuds de type 1 en vert
        "color_2": "rgb(255, 0, 0)", # noeuds de type 2 en rouge
        "color_3": "rgb(0, 0, 255)", # noeuds de type 3 en bleu
        "node_min_size": 1,
        "node_max_size": 10
    }
    final_params = default_params
    if params is not None :
        final_params = {
            *default_params,
            *params
        }
        
    # remplir les dicts
    for datum in data:
        
        if (filter_export_import is not None and datum['export_import'] != filter_export_import):
            continue
            
        if key_1 in datum and key_2 in datum and key_3 in datum:
            value_1 = datum[key_1] if datum[key_1] is not None else "undefined"
            value_2 = datum[key_2] if datum[key_2] is not None else "undefined"
            value_3 = datum[key_2] if datum[key_2] is not None else "undefined"
            value_1_id = key_1 + "_" + value_1
            value_2_id = key_2 + "_" + value_2
            value_3_id = key_3 + "_" + value_3
            
            if value_1_id in key1_uniq:
                key1_uniq[value_1_id] = {**key1_uniq[value_1_id], "size": key1_uniq[value_1_id]["size"] + 1}
            else:
               key1_uniq[value_1_id] = {
                   "type": key_1, 
                   "name": value_1, 
                   "color": final_params["color_1"],
                   "size": 1
               }
            if value_2_id in key2_uniq:
                key2_uniq[value_2_id] = {**key2_uniq[value_2_id], "size": key2_uniq[value_2_id]["size"] + 1}
            else:
               key2_uniq[value_2_id] = {
                   "type": key_2, 
                   "name": value_2, 
                   "color": final_params["color_2"],
                   "size": 1
               }
            if value_3_id in key3_uniq:
                key3_uniq[value_3_id] = {**key3_uniq[value_3_id], "size": key3_uniq[value_3_id]["size"] + 1}
            else:
               key3_uniq[value_3_id] = {
                   "type": key_3, 
                   "name": value_3, 
                   "color": final_params["color_3"],
                   "size": 1
               }
            
            edge_footprints = {
                str(value_1_id + "-" + value_2_id): {
                    'source': value_1_id,
                    'target': value_2_id
                }, 
                str(value_2_id + "-" + value_3_id): {
                    'source': value_2_id,
                    'target': value_3_id
                }, 
                str(value_1_id + "-" + value_3_id): {
                    'source': value_1_id,
                    'target': value_3_id
                }
            } 

            for edge_footprint in edge_footprints.keys():
                if edge_footprint in edges_uniq:
                    edges_uniq[edge_footprint]["weight"] += 1
                else:
                    edges_uniq[edge_footprint] = {
                        "source": edge_footprints[edge_footprint]['source'],
                        "target": edge_footprints[edge_footprint]['target'],
                        "weight": 1
                    }
    # concaténer les deux dicts de noeuds en un seul
    all_nodes = key1_uniq
    all_nodes.update(key2_uniq)
    all_nodes.update(key3_uniq) 

    # applatir et formatter les noeuds
    nodes = []
    for key, node in all_nodes.items():
        nodes.append((key, node))
    edges = []

    for key, edge in edges_uniq.items():
        edges.append((edge["source"], edge["target"], {"weight": edge["weight"]}))

    domain_min_nodes_size = min([node[1]['size'] for node in nodes])
    domain_max_nodes_size = max([node[1]['size'] for node in nodes])
    range_in_nodes_size = [1, 10]
    nodes_size_mapping_params = [domain_min_nodes_size, domain_max_nodes_size, *range_in_nodes_size]

    for node in nodes:
        node[1]["size"] = map_value(node[1]["size"], *nodes_size_mapping_params)
        node[1]["label"] = node[1]["name"]


    Graph.add_nodes_from(nodes)
    Graph.add_edges_from(edges)

    return Sigma(Graph, start_layout=True)

## A. avec filtre sur les imports

In [32]:
render_tripartite_graph(flows, "partner", "product_revolutionempire", "customs_office", filter_export_import = 'Imports')

Sigma(data={'nodes': [('partner_Petites iles', {'type': 'partner', 'name': 'Petites iles', 'color': 'rgb(0, 25…

## B. avec filtre sur les exports

In [33]:
render_tripartite_graph(flows, "partner", "product_revolutionempire", "customs_office", filter_export_import = 'Exports')

Sigma(data={'nodes': [('partner_Petites iles', {'type': 'partner', 'name': 'Petites iles', 'color': 'rgb(0, 25…